In [ ]:
!pip install sqlalchemy

In [ ]:
!pip install pymysql

In [ ]:
pip show sqlalchemy | grep Version

In [ ]:
import numpy as np
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from configparser import RawConfigParser

parser = RawConfigParser()
_ = parser.read('nb.cfg')
engine = create_engine(parser.get('sakila', 'conn_string'))

In [ ]:
def rentals_month(engine, month, year):
    query = f"""
SELECT *
FROM rental
WHERE YEAR(rental_date) = {year}
  AND MONTH(rental_date) = {month};
"""
    df = pd.read_sql_query(query, engine)
    return df
may = rentals_month(engine, 5, 2005)
june = rentals_month(engine, 6, 2005)

In [ ]:
def rental_count_month(df, month, year):
    column_name = f"rentals_{month}_{year}"
    result = df.groupby('customer_id').size().reset_index(name=column_name)
    return result
count_may = rental_count_month(may, 5, 2005)
count_june = rental_count_month(june, 6, 2005)

In [90]:
def compare_rentals(df1, df2):
    month1_col = df1.columns[1] 
    month2_col = df2.columns[1]  
    df_combined = pd.merge(df1, df2, on='customer_id')
    df_combined['difference'] = abs(df_combined[month2_col] - df_combined[month1_col])
    return df_combined
compare_rentals(count_may, count_june)

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1
...,...,...,...,...
507,594,4,6,2
508,595,1,2,1
509,596,6,2,4
510,597,2,3,1
